# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/IP/IP1.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is a

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

Thank you for the warm welcome and the exciting challenge! I'm eager to contribute to this important project and help optimize the radiotherapy treatment plans for our patients.

To solve this problem, I will formulate an integer linear optimization model. The objective is to maximize the total BED scores for all patients while respecting the limited capacity of proton fractions.

Parameters:
- n: The number of patients (n = 17)
- C: The total maximal capacity of proton fractions (C = 100)
- BEDi(j,15-j): The BED score for patient i when receiving j proton fractions and 15-j photon fractions

Decision Variables:
- xij: Binary variable; xij = 1 if patient i receives j proton fractions, and 0 otherwise

Objective Function:
Maximize ∑i=1 to n ∑j=0 to 15 BEDi(j,15-j) * xij

Constraints:
1. Each patient must receive exactly one treatment plan:
   ∑j=0 to 15 xij = 1, for all i = 1 to n

2. The total number of proton fractions used must not exceed the maximal capacity:
   ∑i=1 to n ∑j=0 to 15

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Certainly! Here's a Python code using Pyomo to solve the radiotherapy treatment optimization problem:

```python
import pyomo.environ as pyo
import numpy as np

# Sample data
n = 17  # Number of patients
C = 100  # Total maximal capacity of proton fractions

# Generate sample BED scores (replace with actual data)
np.random.seed(42)
BED = np.random.randint(50, 100, size=(n, 16))

# Create a Pyomo model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(range(n), range(16), domain=pyo.Binary)

# Define objective function
def obj_func(model):
    return sum(BED[i, j] * model.x[i, j] for i in range(n) for j in range(16))
model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define constraints
def one_treatment_plan_rule(model, i):
    return sum(model.x[i, j] for j in range(16)) == 1
model.one_treatment_plan = pyo.Constraint(range(n), rule=one_treatment_plan_rule)

def capacity_rule(model):
    return sum(j * model.x[i, j] for i in range(n) for j in range(

# 4. Input Problem Data and Test Model Code

In [10]:
# Installing pyomo nd solver
!pip install -q pyomo
!pip install pandas
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [12]:
import pyomo.environ as pyo
import pandas as pd

# Sample data
n = 17  # Number of patients
C = 100  # Total maximal capacity of proton fractions

# Generate sample BED scores (replace with actual data)
data = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/IP/IP1/ProblemData.csv", header=None)
BED = pd.DataFrame(data).to_numpy()

# Create a Pyomo model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(range(n), range(16), domain=pyo.Binary)

# Define objective function
def obj_func(model):
    return sum(BED[i, j] * model.x[i, j] for i in range(n) for j in range(16))
model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define constraints
def one_treatment_plan_rule(model, i):
    return sum(model.x[i, j] for j in range(16)) == 1
model.one_treatment_plan = pyo.Constraint(range(n), rule=one_treatment_plan_rule)

def capacity_rule(model):
    return sum(j * model.x[i, j] for i in range(n) for j in range(16)) <= C
model.capacity = pyo.Constraint(rule=capacity_rule)

# Solve the optimization problem
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimization Status:", results.solver.termination_condition)
print("Objective Value:", pyo.value(model.obj))

for i in range(n):
    for j in range(16):
        if pyo.value(model.x[i, j]) == 1:
            print(f"Patient {i+1}: {j} proton fractions, {15-j} photon fractions")

Optimization Status: optimal
Objective Value: 8.239999999999998
Patient 1: 8 proton fractions, 7 photon fractions
Patient 2: 8 proton fractions, 7 photon fractions
Patient 3: 3 proton fractions, 12 photon fractions
Patient 4: 0 proton fractions, 15 photon fractions
Patient 5: 5 proton fractions, 10 photon fractions
Patient 6: 0 proton fractions, 15 photon fractions
Patient 7: 4 proton fractions, 11 photon fractions
Patient 8: 15 proton fractions, 0 photon fractions
Patient 9: 4 proton fractions, 11 photon fractions
Patient 10: 5 proton fractions, 10 photon fractions
Patient 11: 6 proton fractions, 9 photon fractions
Patient 12: 0 proton fractions, 15 photon fractions
Patient 13: 10 proton fractions, 5 photon fractions
Patient 14: 0 proton fractions, 15 photon fractions
Patient 15: 10 proton fractions, 5 photon fractions
Patient 16: 10 proton fractions, 5 photon fractions
Patient 17: 12 proton fractions, 3 photon fractions


# 5. Correct The Model Code to Test Mathematical Model (if applicable)